In [1]:
from IPython.display import display, HTML

# To display the image
display(HTML("<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>"))

# To display the heading and horizontal lines
display(HTML("<h1><hr style='border:none; height:3px;'><center>ML Supervised</center><hr style='border:none; height:3px;'></h1>"))

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Convenience function to create display a progress bar.
# Source : https://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console
def print_progress_bar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()

script_dir = Path.cwd() 
df = pd.read_csv(script_dir.parent / '4 - Dataset' / "regression_weld_data_noOutliers.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Carbon concentration (weight%)                1324 non-null   float64
 1   Silicon concentration (weight%)               1324 non-null   float64
 2   Manganese concentration (weight%)             1324 non-null   float64
 3   Sulphur concentration (weight%)               1324 non-null   float64
 4   Phosphorus concentration (weight%)            1324 non-null   float64
 5   Oxygen concentration (%)                      1324 non-null   float64
 6   Nitrogen concentration (%)                    1324 non-null   float64
 7   Current (A)                                   1324 non-null   float64
 8   Voltage (V)                                   1324 non-null   float64
 9   Heat input (kJ/mm)                            1324 non-null   f

In [13]:

df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Carbon concentration (weight%)                1324 non-null   float64
 1   Silicon concentration (weight%)               1324 non-null   float64
 2   Manganese concentration (weight%)             1324 non-null   float64
 3   Sulphur concentration (weight%)               1324 non-null   float64
 4   Phosphorus concentration (weight%)            1324 non-null   float64
 5   Oxygen concentration (%)                      1324 non-null   float64
 6   Nitrogen concentration (%)                    1324 non-null   float64
 7   Current (A)                                   1324 non-null   float64
 8   Voltage (V)                                   1324 non-null   float64
 9   Heat input (kJ/mm)                            1324 non-null   f

,Carbon concentration (weight%),Silicon concentration (weight%),Manganese concentration (weight%),Sulphur concentration (weight%),Phosphorus concentration (weight%),Oxygen concentration (%),Nitrogen concentration (%),Current (A),Voltage (V),Heat input (kJ/mm),...,AC or DC,Electrode positive or negative,Type of weld,Yield strength (MPa),Ultimate tensile strength (MPa),Elongation (%),Reduction of Area (%),Charpy temperature (deg C),Charpy impact toughness (J),Hardness (kg/mm2)
0,0.042,0.28,1.37,0.007,0.012,0.0421,0.0082,170.0,21.0,2.2,...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,-57.0,100.0,NaN
1,0.075,0.35,1.09,0.006,0.008,0.0424,0.0087,170.0,21.0,1.0,...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,-57.0,100.0,NaN
2,0.090,0.59,1.10,0.013,0.020,0.0540,0.0082,150.0,37.5,1.0,...,0.0,1.0,0.0,648.0,743.0,18.0,58.0,NaN,NaN,247.0
3,0.078,0.43,1.10,0.007,0.005,0.0329,0.0073,170.0,21.0,1.0,...,0.0,1.0,0.0,399.0,492.0,32.2,80.7,-56.0,100.0,NaN
4,0.120,0.14,1.03,0.002,0.006,0.0166,0.0022,500.0,29.0,2.9,...,1.0,0.0,1.0,NaN,745.0,NaN,NaN,-20.0,172.0,NaN


## Création X et y

In [14]:
L_features = [
    'Carbon concentration (weight%)', 'Silicon concentration (weight%)',
       'Manganese concentration (weight%)', 'Sulphur concentration (weight%)',
       'Phosphorus concentration (weight%)', 'Nickel concentration (weight%)',
       'Chromium concentration (weight%)',
       'Molybdenum concentration (weight%)',
       'Vanadium concentration (weight%)', 'Copper concentration (weight%)',
       'Oxygen concentration (%)', 'Titanium concentration (%)',
       'Nitrogen concentration (%)', 'Aluminium concentration (%)',
       'Boron concentration (%)', 'Niobium concentration (%)', 'Current (A)',
       'Voltage (V)', 'AC or DC', 'Electrode positive or negative',
       'Heat input (kJ/mm)', 'Interpass temperature (deg C)', 'Type of weld',
       'Post weld heat treatment temperature (deg C)',
       'Post weld heat treatment time (hours)', 'Charpy temperature (deg C)'
]


L_targets = [
    'Yield strength (MPa)', 
       'Ultimate tensile strength (MPa)', 'Elongation (%)',
       'Reduction of Area (%)', 'Charpy impact toughness (J)', 'Hardness (kg/mm2)'
]


X = df.loc[:, L_features]
ys = df.loc[:,L_targets]
print(ys.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Yield strength (MPa)             1003 non-null   float64
 1   Ultimate tensile strength (MPa)  919 non-null    float64
 2   Elongation (%)                   853 non-null    float64
 3   Reduction of Area (%)            828 non-null    float64
 4   Charpy impact toughness (J)      806 non-null    float64
 5   Hardness (kg/mm2)                100 non-null    float64
dtypes: float64(6)
memory usage: 62.2 KB
None


In [15]:
from sklearn.model_selection import train_test_split

def trainTest(X,y) :
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Fonction pour calculer le RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Fonction pour calculer le R² ajusté
def calculate_adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Configuration de la validation croisée (5 folds)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Fonction pour entraîner et évaluer le modèle Bayesian Ridge
def bayesian_ridge_regression(X_train, X_test, y_train, y_test):
    model = BayesianRidge()

    # Validation croisée sur l'ensemble d'entraînement
    y_train_pred_cv = cross_val_predict(model, X_train, y_train, cv=kf)

    # Ajustement du modèle et prédictions sur l'ensemble de test
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)

    # Calcul des scores
    rmse_train = calculate_rmse(y_train, y_train_pred_cv)
    rmse_test = calculate_rmse(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred_cv)
    r2_test = r2_score(y_test, y_test_pred)

    # Nombre d'observations et de prédicteurs
    n_train, p_train = X_train.shape

    return {
        "Model": "Bayesian Ridge",
        "RMSE_Train": rmse_train,
        "RMSE_Test": rmse_test,
        "R2_Train": r2_train,
        "R2_Test": r2_test,
        "Adjusted_R2_Train": calculate_adjusted_r2(r2_train, n_train, p_train),
        "Adjusted_R2_Test": calculate_adjusted_r2(r2_test, len(y_test), 1),  # 1 car on a 1 seul prédicteur
        "Hyperparameters": model.get_params()
    }

# Fonction pour entraîner et évaluer le modèle SVR avec Random Grid Search
def svr_regression(X_train, X_test, y_train, y_test):
    model = SVR()
    param_grid = {
        'kernel': ['rbf', 'poly'],
        'C': np.logspace(-3, 3, 7),
        'gamma': ['scale', 'auto'],
        'degree': [2, 3, 4]
    }
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=4, cv=kf, random_state=42)
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_

    # Validation croisée sur l'ensemble d'entraînement
    y_train_pred_cv = cross_val_predict(best_model, X_train, y_train, cv=kf)

    # Prédictions sur l'ensemble de test
    y_test_pred = best_model.predict(X_test)

    # Calcul des scores
    rmse_train = calculate_rmse(y_train, y_train_pred_cv)
    rmse_test = calculate_rmse(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred_cv)
    r2_test = r2_score(y_test, y_test_pred)

    # Nombre d'observations et de prédicteurs
    n_train, p_train = X_train.shape

    return {
        "Model": "SVR",
        "RMSE_Train": rmse_train,
        "RMSE_Test": rmse_test,
        "R2_Train": r2_train,
        "R2_Test": r2_test,
        "Adjusted_R2_Train": calculate_adjusted_r2(r2_train, n_train, p_train),
        "Adjusted_R2_Test": calculate_adjusted_r2(r2_test, len(y_test), 1),  # 1 car on a 1 seul prédicteur
        "Hyperparameters": best_model.get_params()
    }

# Fonction pour entraîner et évaluer le modèle de Gradient Boosting avec Random Grid Search
def gradient_boosting_regression(X_train, X_test, y_train, y_test):
    model = GradientBoostingRegressor()
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.05],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 5, 10]
    }
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=4, cv=kf, random_state=42)
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_

    # Validation croisée sur l'ensemble d'entraînement
    y_train_pred_cv = cross_val_predict(best_model, X_train, y_train, cv=kf)

    # Prédictions sur l'ensemble de test
    y_test_pred = best_model.predict(X_test)

    # Calcul des scores
    rmse_train = calculate_rmse(y_train, y_train_pred_cv)
    rmse_test = calculate_rmse(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred_cv)
    r2_test = r2_score(y_test, y_test_pred)

    # Nombre d'observations et de prédicteurs
    n_train, p_train = X_train.shape

    return {
        "Model": "Gradient Boosting",
        "RMSE_Train": rmse_train,
        "RMSE_Test": rmse_test,
        "R2_Train": r2_train,
        "R2_Test": r2_test,
        "Adjusted_R2_Train": calculate_adjusted_r2(r2_train, n_train, p_train),
        "Adjusted_R2_Test": calculate_adjusted_r2(r2_test, len(y_test), 1),  # 1 car on a 1 seul prédicteur
        "Hyperparameters": best_model.get_params()
    }

# Fonction pour entraîner et évaluer le modèle de Forêt Aléatoire avec Random Grid Search
def random_forest_regression(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=4, cv=kf, random_state=42)
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_

    # Validation croisée sur l'ensemble d'entraînement
    y_train_pred_cv = cross_val_predict(best_model, X_train, y_train, cv=kf)

    # Prédictions sur l'ensemble de test
    y_test_pred = best_model.predict(X_test)

    # Calcul des scores
    rmse_train = calculate_rmse(y_train, y_train_pred_cv)
    rmse_test = calculate_rmse(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred_cv)
    r2_test = r2_score(y_test, y_test_pred)

    # Nombre d'observations et de prédicteurs
    n_train, p_train = X_train.shape

    return {
        "Model": "Random Forest",
        "RMSE_Train": rmse_train,
        "RMSE_Test": rmse_test,
        "R2_Train": r2_train,
        "R2_Test": r2_test,
        "Adjusted_R2_Train": calculate_adjusted_r2(r2_train, n_train, p_train),
        "Adjusted_R2_Test": calculate_adjusted_r2(r2_test, len(y_test), 1),  # 1 car on a 1 seul prédicteur
        "Hyperparameters": best_model.get_params()
    }

# Fonction pour entraîner et évaluer la régression linéaire
def linear_regression(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Validation croisée sur l'ensemble d'entraînement
    y_train_pred_cv = cross_val_predict(model, X_train, y_train, cv=kf)

    # Prédictions sur l'ensemble de test
    y_test_pred = model.predict(X_test)

    # Calcul des scores
    rmse_train = calculate_rmse(y_train, y_train_pred_cv)
    rmse_test = calculate_rmse(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred_cv)
    r2_test = r2_score(y_test, y_test_pred)

    # Nombre d'observations et de prédicteurs
    n_train, p_train = X_train.shape

    return {
        "Model": "Linear Regression",
        "RMSE_Train": rmse_train,
        "RMSE_Test": rmse_test,
        "R2_Train": r2_train,
        "R2_Test": r2_test,
        "Adjusted_R2_Train": calculate_adjusted_r2(r2_train, n_train, p_train),
        "Adjusted_R2_Test": calculate_adjusted_r2(r2_test, len(y_test), 1),  # 1 car on a 1 seul prédicteur
        "Hyperparameters": model.get_params()
    }

# Fonction pour entraîner et évaluer la régression Ridge avec Random Grid Search
def ridge_regression(X_train, X_test, y_train, y_test):
    model = Ridge()
    param_grid = {
        'alpha': [0.01, 0.1, 1, 10, 100]
    }
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=4, cv=kf, random_state=42)
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_

    # Validation croisée sur l'ensemble d'entraînement
    y_train_pred_cv = cross_val_predict(best_model, X_train, y_train, cv=kf)

    # Prédictions sur l'ensemble de test
    y_test_pred = best_model.predict(X_test)

    # Calcul des scores
    rmse_train = calculate_rmse(y_train, y_train_pred_cv)
    rmse_test = calculate_rmse(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred_cv)
    r2_test = r2_score(y_test, y_test_pred)

    # Nombre d'observations et de prédicteurs
    n_train, p_train = X_train.shape

    return {
        "Model": "Ridge Regression",
        "RMSE_Train": rmse_train,
        "RMSE_Test": rmse_test,
        "R2_Train": r2_train,
        "R2_Test": r2_test,
        "Adjusted_R2_Train": calculate_adjusted_r2(r2_train, n_train, p_train),
        "Adjusted_R2_Test": calculate_adjusted_r2(r2_test, len(y_test), 1),  # 1 car on a 1 seul prédicteur
        "Hyperparameters": best_model.get_params()
    }

# Fonction pour entraîner et évaluer la régression Lasso avec Random Grid Search
def lasso_regression(X_train, X_test, y_train, y_test):
    model = Lasso()
    param_grid = {
        'alpha': [0.01, 0.1, 1, 10, 100]
    }
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=4, cv=kf, random_state=42)
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_

    # Validation croisée sur l'ensemble d'entraînement
    y_train_pred_cv = cross_val_predict(best_model, X_train, y_train, cv=kf)

    # Prédictions sur l'ensemble de test
    y_test_pred = best_model.predict(X_test)

    # Calcul des scores
    rmse_train = calculate_rmse(y_train, y_train_pred_cv)
    rmse_test = calculate_rmse(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred_cv)
    r2_test = r2_score(y_test, y_test_pred)

    # Nombre d'observations et de prédicteurs
    n_train, p_train = X_train.shape

    return {
        "Model": "Lasso Regression",
        "RMSE_Train": rmse_train,
        "RMSE_Test": rmse_test,
        "R2_Train": r2_train,
        "R2_Test": r2_test,
        "Adjusted_R2_Train": calculate_adjusted_r2(r2_train, n_train, p_train),
        "Adjusted_R2_Test": calculate_adjusted_r2(r2_test, len(y_test), 1),  # 1 car on a 1 seul prédicteur
        "Hyperparameters": best_model.get_params()
    }


# Fonction principale qui exécute tous les modèles et retourne les résultats dans un DataFrame
def evaluate_all_models(X_train, X_test, y_train, y_test):
    results = []

    # Appel des fonctions de régression et stockage des résultats
    results.append(bayesian_ridge_regression(X_train, X_test, y_train, y_test))
    results.append(svr_regression(X_train, X_test, y_train, y_test))
    results.append(gradient_boosting_regression(X_train, X_test, y_train, y_test))
    results.append(random_forest_regression(X_train, X_test, y_train, y_test))
    results.append(linear_regression(X_train, X_test, y_train, y_test))
    results.append(ridge_regression(X_train, X_test, y_train, y_test))
    results.append(lasso_regression(X_train, X_test, y_train, y_test))
    # Conversion des résultats en DataFrame
    return pd.DataFrame(results)


def evaluateAllTarget(df, Features, Targets):
    progress = 0
    results_dict = {}  # Dictionnaire pour stocker les DataFrames

    # Display a progress bar
    print_progress_bar(progress, int(len(Targets)), prefix='Progress:', suffix='Complete', length=50)

    for i in range(len(Targets)):
        progress += 1
        print_progress_bar(progress, int(len(Targets)), prefix='Progress:', suffix='Complete', length=50)

        col_name = Targets[i]

        # Filtrer les colonnes selon la condition
        if Targets[i] == 'Charpy impact toughness (J)':
            df_weld = df.loc[:, Features + [Targets[i]]]
        else:
            df_weld = df.loc[:, Features + [Targets[i]]]
            df_weld = df_weld.drop('Charpy temperature (deg C)', axis=1)

        df_weld = df_weld.dropna()  # Supprimer les lignes avec des valeurs manquantes
        print(f"\n\nTaille du dataset avant drop duplicates : {col_name} : {df_weld.shape}")
        
        df_weld = df_weld.drop_duplicates()  # Supprimer les doublons
        X = df_weld.drop(Targets[i], axis=1)  # Features
        y = df_weld.loc[:, Targets[i]]  # Target

        print(f"Taille du dataset avec target {col_name} : {df_weld.shape}")

        # Diviser en ensemble d'entraînement et de test
        X_train, X_test, y_train, y_test = train_test_split(X, y)

        print(f"Résultat pour {col_name} avec pour moyenne {y.mean()}: ")

        # Évaluer tous les modèles et obtenir un DataFrame des résultats
        df_results = evaluate_all_models(X_train, X_test, y_train, y_test)

        # Stocker df_results dans le dictionnaire avec col_name comme clé
        results_dict[col_name] = df_results

        print(df_results)

    return results_dict  # Retourner le dictionnaire de DataFrames

results_dict = evaluateAllTarget(df, L_features, L_targets)

Progress: |████████------------------------------------------| 16.7% Complete

Taille du dataset avant drop duplicates : Yield strength (MPa) : (1003, 26)
Taille du dataset avec target Yield strength (MPa) : (962, 26)
Résultat pour Yield strength (MPa) avec pour moyenne 497.7261954261954: 
               Model  RMSE_Train  RMSE_Test  R2_Train   R2_Test  \
0     Bayesian Ridge   77.701511  83.525924  0.197058  0.257785   
1                SVR   74.806022  81.537793  0.255785  0.292698   
2  Gradient Boosting   44.185399  44.138226  0.740353  0.792739   
3      Random Forest   49.807220  51.037186  0.670079  0.722885   
4  Linear Regression   66.764651  74.230182  0.407186  0.413797   
5   Ridge Regression   67.806379  73.579684  0.388543  0.424026   
6   Lasso Regression   66.313659  73.721410  0.415168  0.421805   

   Adjusted_R2_Train  Adjusted_R2_Test  \
0           0.168176          0.254680   
1           0.229015          0.289739   
2           0.731014          0.791872   
3   

In [17]:
results_dict['Yield strength (MPa)'].sort_values(by='R2_Test', ascending=False)

,Model,RMSE_Train,RMSE_Test,R2_Train,R2_Test,Adjusted_R2_Train,Adjusted_R2_Test,Hyperparameters
2,Gradient Boosting,44.185399,44.138226,0.740353,0.792739,0.731014,0.791872,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
3,Random Forest,49.807220,51.037186,0.670079,0.722885,0.658212,0.721725,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
5,Ridge Regression,67.806379,73.579684,0.388543,0.424026,0.366548,0.421616,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
6,Lasso Regression,66.313659,73.721410,0.415168,0.421805,0.394131,0.419386,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
4,Linear Regression,66.764651,74.230182,0.407186,0.413797,0.385862,0.411344,"{'copy_X': True, 'fit_intercept': True, 'n_job..."
1,SVR,74.806022,81.537793,0.255785,0.292698,0.229015,0.289739,"{'C': 10.0, 'cache_size': 200, 'coef0': 0.0, '..."
0,Bayesian Ridge,77.701511,83.525924,0.197058,0.257785,0.168176,0.254680,"{'alpha_1': 1e-06, 'alpha_2': 1e-06, 'alpha_in..."


In [18]:
results_dict['Charpy impact toughness (J)'].sort_values(by='R2_Test', ascending=False)

,Model,RMSE_Train,RMSE_Test,R2_Train,R2_Test,Adjusted_R2_Train,Adjusted_R2_Test,Hyperparameters
3,Random Forest,27.626072,22.967608,0.692503,0.805284,0.678623,0.804306,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,Gradient Boosting,26.822698,24.460399,0.710127,0.779150,0.697042,0.778041,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
6,Lasso Regression,33.891949,36.277185,0.537197,0.514223,0.516307,0.511782,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
5,Ridge Regression,34.516846,36.421665,0.519974,0.510346,0.498306,0.507886,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
4,Linear Regression,33.655437,37.054320,0.543634,0.493188,0.523034,0.490641,"{'copy_X': True, 'fit_intercept': True, 'n_job..."
0,Bayesian Ridge,38.817629,37.897220,0.392899,0.469868,0.365495,0.467204,"{'alpha_1': 1e-06, 'alpha_2': 1e-06, 'alpha_in..."
1,SVR,39.562539,38.260974,0.369375,0.459642,0.340909,0.456927,"{'C': 1000.0, 'cache_size': 200, 'coef0': 0.0,..."


In [19]:
results_dict['Ultimate tensile strength (MPa)'].sort_values(by='R2_Test', ascending=False)

,Model,RMSE_Train,RMSE_Test,R2_Train,R2_Test,Adjusted_R2_Train,Adjusted_R2_Test,Hyperparameters
2,Gradient Boosting,47.845493,42.491747,0.725685,0.753864,0.714834,0.752735,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
3,Random Forest,51.689748,44.263747,0.679834,0.732907,0.667169,0.731682,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
4,Linear Regression,65.756492,59.163581,0.481864,0.522828,0.461368,0.520639,"{'copy_X': True, 'fit_intercept': True, 'n_job..."
6,Lasso Regression,65.740647,59.827539,0.482113,0.512058,0.461627,0.509820,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
5,Ridge Regression,66.474519,61.063581,0.470486,0.491688,0.449540,0.489356,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
1,SVR,70.727396,65.014112,0.400565,0.423789,0.376853,0.421146,"{'C': 10.0, 'cache_size': 200, 'coef0': 0.0, '..."
0,Bayesian Ridge,75.658146,73.951537,0.314072,0.254478,0.286939,0.251058,"{'alpha_1': 1e-06, 'alpha_2': 1e-06, 'alpha_in..."


In [9]:
results_dict['Charpy impact toughness (J)'].sort_values(by='R2_Test', ascending=False)

,Model,RMSE_Train,RMSE_Test,R2_Train,R2_Test,Adjusted_R2_Train,Adjusted_R2_Test,Hyperparameters
3,Random Forest,25.441196,25.241548,0.736814,0.760525,0.725952,0.759427,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,Gradient Boosting,25.239356,27.457379,0.740973,0.716635,0.730283,0.715336,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
4,Linear Regression,33.542138,37.734147,0.542522,0.464824,0.523642,0.462369,"{'copy_X': True, 'fit_intercept': True, 'n_job..."
6,Lasso Regression,33.829283,37.966905,0.534656,0.458202,0.515451,0.455716,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
5,Ridge Regression,34.176537,38.131062,0.525054,0.453506,0.505453,0.450999,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
1,SVR,37.431188,39.720152,0.430288,0.407008,0.406776,0.404287,"{'C': 10.0, 'cache_size': 200, 'coef0': 0.0, '..."
0,Bayesian Ridge,37.448530,40.530645,0.429760,0.382560,0.406226,0.379728,"{'alpha_1': 1e-06, 'alpha_2': 1e-06, 'alpha_in..."
